In [ ]:
# default_exp core

In [ ]:
#hide
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#hide
%cd "/content/drive/MyDrive/Coding/ModelAssistedLabel"

/content/drive/MyDrive/Coding/ModelAssistedLabel


# Trainer
> Class wrapper for train.py

In [ ]:
from ModelAssistedLabel.core import Defaults

In [ ]:
#export
import os

class Trainer():
  "Write the backbone of the model to file and then run YOLOv5's train file."

  def __init__(self, name, yaml_file = "./yolov5/models/custom_yolov5s.yaml"):
    """
    sets the current directory to the project's root as defined in Defaults.

    Args:
      name: identifier for results
      yaml_file: path to write the file 
    """
    os.chdir(Defaults().root)
    self.yaml_file = yaml_file
    self.name = name
    self.template = Defaults().trainer_template

  def write_yaml(self):
    """
    Reads the number of classes from `root`/data.yaml and sets up YOLOv5.
    """
    import yaml
    with open("data.yaml", 'r') as stream:
      self.num_classes = str(yaml.safe_load(stream)['nc'])
    if os.path.exists(self.yaml_file):
      os.remove(self.yaml_file)

    f = open(self.yaml_file,"w+")
    f.writelines(self.template)
    f.close()

  def train(self, epochs):
    """
    wrapper for train.py.

    Args:
      epochs: number of iterations
    """
    self.write_yaml()
    os.chdir("yolov5")
    yaml_file = self.yaml_file.replace("yolov5/", "") #accounting for change in relative path
    os.system(f"python train.py --img 416 --batch 16 --epochs {epochs} --data '../data.yaml' --cfg {yaml_file} --weights '' --name {self.name}  --cache")
    os.chdir("..")